In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import types
from functools import partial

In [3]:
import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from torch.utils.data import DataLoader, random_split, Subset
from IPython.display import clear_output
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

In [4]:
from lared_laughter.fusion.dataset import FatherDataset, LabelExtractor
from lared_laughter.accel.dataset import AccelExtractor
from lared_laughter.audio.dataset import AudioLaughterExtractor
from lared_laughter.video.dataset import VideoExtractor
from lared_laughter.video.dataset.transforms import get_kinetics_val_transform
from lared_laughter.constants import annot_exp_path, datasets_path
from lared_laughter.utils import get_metrics, load_examples
from lared_laughter.fusion.model import FusionModel
from lared_laughter.accel.system import System as AccelSystem

In [5]:
examples = load_examples(os.path.join(annot_exp_path, 'processed', 'examples_without_calibration.csv'))

In [84]:
extractors = {}
# audios_path = os.path.join(datasets_path, "loose", "lared_audios.pkl")
# extractors['audio'] = {
#     'extractor': AudioLaughterExtractor(audios_path), 
#     'id_column': 'hash'
# }
# videos_path = os.path.join(datasets_path, 'loose', 'video')
# extractors['video'] = {
#     'extractor': VideoExtractor(videos_path,
#         transform=get_kinetics_val_transform(8, 256, False)), 
#     'id_column': 'hash'
# }
accel_ds_path = os.path.join(datasets_path, 'loose', 'accel_long.pkl')
extractors['accel'] = {
    'extractor': AccelExtractor(accel_ds_path, min_len=30, max_len=30), 
    'id_column': 'hash'
}
ds = FatherDataset(examples, extractors, label_column='intensity', id_column='hash', eval=False)

In [85]:
ds[100]['intval']

[3.044595156848411, 4.544595156848411]

In [86]:
examples[['onset_times', 'offset_times']].iloc[101,:]

onset_times     []
offset_times    []
Name: 101, dtype: object

In [105]:
ds[101]

{'accel': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 'label': 1.0,
 'intval': [5.202356235357091, 6.210401340571025],
 'index': 101}